In [94]:
from lxml import html
import requests
import nltk
from pyvi import ViTokenizer
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from gensim.models import KeyedVectors 

In [95]:
url ="https://vnexpress.net/tin-tuc/the-gioi/indonesia-tim-thay-cang-dap-va-mot-doan-than-may-bay-roi-xuong-bien-3833096.html"

In [96]:
page = requests.get(url)

In [97]:
data = page.content.decode('UTF-8')
node = html.fromstring(data)

In [98]:
buyers = node.xpath('//p[@class="Normal"]/text()')

In [99]:
buyers

['\nCơ quan Tìm kiếm Cứu hộ Quốc gia Indonesia (Basarnas) hôm qua cho biết đã tìm thấy cụm càng đáp, bánh xe và một đoạn lớn của thân máy bay Boeing 737 MAX 8 bị rơi hồi đầu tuần. Cơ quan này cũng áp dụng chiến thuật mới để tìm kiếm các phần thân phi cơ đang nằm dưới đáy biển, ',
 'đưa tin.',
 '\nNhà chức trách Indonesia đang tận dụng tàu thăm dò thuộc một công ty dầu khí, do thủy thủ đoàn đã rất quen thuộc với địa hình đáy biển ở nơi máy bay gặp nạn. Tàu lặn không người lái cũng được triển khai liên tục. Điều này có thể giúp nhanh chóng phát hiện những vật thể lạ dưới đáy biển như mảnh xác phi cơ.',
 '\nCác thợ lặn của Basarnas, hải quân và cảnh sát Indonesia đang tích cực tìm kiếm những bộ phận máy bay dưới đáy biển. Nhà chức trách Indonesia hôm nay dự kiến điều một tàu lắp cần cẩu tới trục vớt các mảnh xác phi cơ.',
 '\nGiám đốc Basarnas Muhammad Syaugi tiết lộ hộp đen được tìm thấy trước đó là thiết bị ghi tham số chuyến bay. Tuy nhiên, họ chưa phát hiện tín hiệu định vị từ thiết b

In [69]:
data_string =""
for i in range(len(buyers)):
    data_string+=buyers[i]

In [70]:
data_string

'\nCơ quan Tìm kiếm Cứu hộ Quốc gia Indonesia (Basarnas) hôm qua cho biết đã tìm thấy cụm càng đáp, bánh xe và một đoạn lớn của thân máy bay Boeing 737 MAX 8 bị rơi hồi đầu tuần. Cơ quan này cũng áp dụng chiến thuật mới để tìm kiếm các phần thân phi cơ đang nằm dưới đáy biển, đưa tin.\nNhà chức trách Indonesia đang tận dụng tàu thăm dò thuộc một công ty dầu khí, do thủy thủ đoàn đã rất quen thuộc với địa hình đáy biển ở nơi máy bay gặp nạn. Tàu lặn không người lái cũng được triển khai liên tục. Điều này có thể giúp nhanh chóng phát hiện những vật thể lạ dưới đáy biển như mảnh xác phi cơ.\nCác thợ lặn của Basarnas, hải quân và cảnh sát Indonesia đang tích cực tìm kiếm những bộ phận máy bay dưới đáy biển. Nhà chức trách Indonesia hôm nay dự kiến điều một tàu lắp cần cẩu tới trục vớt các mảnh xác phi cơ.\nGiám đốc Basarnas Muhammad Syaugi tiết lộ hộp đen được tìm thấy trước đó là thiết bị ghi tham số chuyến bay. Tuy nhiên, họ chưa phát hiện tín hiệu định vị từ thiết bị ghi âm buồng lái, n

In [100]:
data_string = data_string.lower()
data_string = data_string.replace('\n',"")
data_string = data_string.strip()
data_string

'cơ quan tìm kiếm cứu hộ quốc gia indonesia (basarnas) hôm qua cho biết đã tìm thấy cụm càng đáp, bánh xe và một đoạn lớn của thân máy bay boeing 737 max 8 bị rơi hồi đầu tuần. cơ quan này cũng áp dụng chiến thuật mới để tìm kiếm các phần thân phi cơ đang nằm dưới đáy biển, đưa tin.nhà chức trách indonesia đang tận dụng tàu thăm dò thuộc một công ty dầu khí, do thủy thủ đoàn đã rất quen thuộc với địa hình đáy biển ở nơi máy bay gặp nạn. tàu lặn không người lái cũng được triển khai liên tục. điều này có thể giúp nhanh chóng phát hiện những vật thể lạ dưới đáy biển như mảnh xác phi cơ.các thợ lặn của basarnas, hải quân và cảnh sát indonesia đang tích cực tìm kiếm những bộ phận máy bay dưới đáy biển. nhà chức trách indonesia hôm nay dự kiến điều một tàu lắp cần cẩu tới trục vớt các mảnh xác phi cơ.giám đốc basarnas muhammad syaugi tiết lộ hộp đen được tìm thấy trước đó là thiết bị ghi tham số chuyến bay. tuy nhiên, họ chưa phát hiện tín hiệu định vị từ thiết bị ghi âm buồng lái, nhiều khả

In [101]:
sentences = nltk.sent_tokenize(data_string)
sentences

['cơ quan tìm kiếm cứu hộ quốc gia indonesia (basarnas) hôm qua cho biết đã tìm thấy cụm càng đáp, bánh xe và một đoạn lớn của thân máy bay boeing 737 max 8 bị rơi hồi đầu tuần.',
 'cơ quan này cũng áp dụng chiến thuật mới để tìm kiếm các phần thân phi cơ đang nằm dưới đáy biển, đưa tin.nhà chức trách indonesia đang tận dụng tàu thăm dò thuộc một công ty dầu khí, do thủy thủ đoàn đã rất quen thuộc với địa hình đáy biển ở nơi máy bay gặp nạn.',
 'tàu lặn không người lái cũng được triển khai liên tục.',
 'điều này có thể giúp nhanh chóng phát hiện những vật thể lạ dưới đáy biển như mảnh xác phi cơ.các thợ lặn của basarnas, hải quân và cảnh sát indonesia đang tích cực tìm kiếm những bộ phận máy bay dưới đáy biển.',
 'nhà chức trách indonesia hôm nay dự kiến điều một tàu lắp cần cẩu tới trục vớt các mảnh xác phi cơ.giám đốc basarnas muhammad syaugi tiết lộ hộp đen được tìm thấy trước đó là thiết bị ghi tham số chuyến bay.',
 'tuy nhiên, họ chưa phát hiện tín hiệu định vị từ thiết bị ghi âm

In [102]:
w2v = KeyedVectors.load_word2vec_format("vi_txt/vi.vec")

In [103]:
vocab = w2v.wv.vocab 
X = []
for sentence in sentences:
    sentence_tokenized = ViTokenizer.tokenize(sentence)
    words = sentence_tokenized.split(" ")
    sentence_vec = np.zeros((100))
    for word in words:
        if word in vocab:
            sentence_vec+=w2v.wv[word]
    X.append(sentence_vec)
X

C:\Users\TYE\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
C:\Users\TYE\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


[array([ 5.39630987e-01,  3.04471300e+00, -4.79527568e+00, -1.38958207e+00,
         2.17558600e+00,  1.16677501e+00, -1.88026759e+00, -2.57608982e-01,
        -2.44154594e+00,  5.55632298e+00, -4.88880869e-01, -6.19116004e-01,
        -6.07304048e-01, -6.43640902e+00, -6.81311301e+00,  3.37716600e+00,
        -4.29821500e+00, -3.24947358e+00,  6.17568597e+00, -4.21552099e+00,
         1.42312502e-02,  4.01342106e+00,  7.84582959e-01, -1.07912373e-01,
        -2.93844659e+00,  2.07515621e+00,  1.63445539e+00,  9.65158923e-01,
         1.77336593e+00, -1.46778196e+00, -3.54007297e+00,  5.30094963e-01,
         1.02474284e+00,  1.37305958e+00,  2.78254350e+00, -2.97154052e+00,
        -3.39838046e+00,  1.44554381e+00, -3.61620888e+00, -4.15787299e+00,
         2.13067486e+00, -1.84250706e+00, -3.57863306e+00, -2.16258005e+00,
         1.29167312e+00, -1.06631221e+01, -6.93371799e-03, -7.28275000e+00,
         9.65801985e-01,  8.78061017e-01, -2.22649707e+00, -5.01375989e+00,
         6.9

In [104]:
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(X)

In [105]:
avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = ' '.join([sentences[closest[idx]] for idx in ordering])

In [106]:
summary

'tàu lặn không người lái cũng được triển khai liên tục. cơ quan này cũng áp dụng chiến thuật mới để tìm kiếm các phần thân phi cơ đang nằm dưới đáy biển, đưa tin.nhà chức trách indonesia đang tận dụng tàu thăm dò thuộc một công ty dầu khí, do thủy thủ đoàn đã rất quen thuộc với địa hình đáy biển ở nơi máy bay gặp nạn. nhà chức trách indonesia hôm nay dự kiến điều một tàu lắp cần cẩu tới trục vớt các mảnh xác phi cơ.giám đốc basarnas muhammad syaugi tiết lộ hộp đen được tìm thấy trước đó là thiết bị ghi tham số chuyến bay. lãnh đạo ủy ban an toàn giao thông quốc gia indonesia nhận định phải mất khoảng ba tuần để tải dữ liệu và thêm 6 tháng để phân tích hộp đen.chuyến bay jt610 của hãng lion air rơi xuống biển sáng 29/10, không lâu sau khi cất cánh từ thủ đô jakarta, khiến toàn bộ 189 người trên máy bay thiệt mạng. máy bay gặp nạn là một chiếc boeing 737 max 8 mới được chuyển giao cho lion air hồi tháng 8 năm nay.đại diện lion air cho biết phi công đã thông báo máy bay gặp vấn đề kỹ thuậ